In [7]:
import os
import json
import yaml
import geopandas as gpd
from shapely.geometry import shape
from labelbox import Client


def download_labels(config_path, study_site, output_root):
    # Load config
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    api_key = config["api_key"]
    project_id = config["project_id"]

    client = Client(api_key)
    project = client.get_project(project_id)
    print(f"Connected to project: {project.name}")

    # Kick off export
    export_task = project.export_v2(params={
        "attachments": False,
        "metadata_fields": False,
        "data_row_details": True,
        "label_details": True,
    })

    print("Waiting for export to complete...")
    export_task.wait_until_done()

    if export_task.errors:
        raise Exception(f"Export failed: {export_task.errors}")

    # Export result is inline here (list of dicts)
    export_result = export_task.result
    print(f"Received {len(export_result)} rows from export")

    features = []

    for record in export_result:
        row_name = record.get("data_row", {}).get("global_key", "")
        if not row_name.startswith(study_site):
            continue

        # navigate into the project block then labels
        projects = record.get("projects", {})
        for proj_id, proj_content in projects.items():
            for label in proj_content.get("labels", []):
                annotations = label.get("annotations", {}).get("objects", [])
                for ann in annotations:
                    if ann.get("annotation_kind") in ("GeoPolygon", "GeoPolyline"):
                        geom = ann["geojson"]
                        properties = {
                            "data_row_id": record.get("data_row", {}).get("id"),
                            "name": row_name,
                            "feature_id": ann.get("feature_id"),
                            "class_name": ann.get("name"),
                            "study_site": study_site
                        }
                        features.append({
                            "geometry": shape(geom),
                            "properties": properties
                        })

    if not features:
        raise Exception("No matching features found in export.")

    # Build GeoDataFrame
    geoms = [f["geometry"] for f in features]
    props = [f["properties"] for f in features]
    gdf = gpd.GeoDataFrame(props, geometry=geoms, crs="EPSG:4326")

    # Output path
    out_dir = os.path.join(
        output_root,
        f"{study_site} 50x50 km - PLD",
        f"{study_site} Lakes from Labelbox - Shapefile"
    )
    os.makedirs(out_dir, exist_ok=True)

    # Save shapefile path
    shp_path = os.path.join(out_dir, f"{study_site}_lakes.shp")
    gdf.to_file(shp_path, driver="ESRI Shapefile")
    print(f"Saved shapefile: {shp_path}")

    # Return shapefile path
    return shp_path


if __name__ == "__main__":
    config_path = r"D:\planetscope_lake_ice\labelbox_water_body_delineation_config.yaml"
    study_site = "GR"
    output_root = r"E:\planetscope_lake_ice\Data\Input"

    shp_path = download_labels(config_path, study_site, output_root)
    print("Returned shapefile path:", shp_path)

Connected to project: Lake Ice Project - Water Body Delineation


C:\Users\nj142\AppData\Local\Temp\ipykernel_40408\109699664.py:21: DeprecationWarning: You are currently utilizing export_v2 for this action, which will be removed in 7.0. Please refer to our docs for export alternatives. https://docs.labelbox.com/reference/export-overview#export-methods
  export_task = project.export_v2(params={


Waiting for export to complete...
Received 3 rows from export
Saved shapefile: E:\planetscope_lake_ice\Data\Input\GR 50x50 km - PLD\GR Lakes from Labelbox - Shapefile\GR_lakes.shp
Returned shapefile path: E:\planetscope_lake_ice\Data\Input\GR 50x50 km - PLD\GR Lakes from Labelbox - Shapefile\GR_lakes.shp


C:\Users\nj142\AppData\Local\Temp\ipykernel_40408\109699664.py:83: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(shp_path, driver="ESRI Shapefile")
c:\Users\nj142\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'data_row_id' to 'data_row_i'
  ogr_write(


In [8]:
import os
import json
import yaml
import geopandas as gpd
from shapely.geometry import shape
from labelbox import Client


def latlon_to_utm_epsg(lat, lon):
    """
    Return UTM EPSG code (WGS84) for a given latitude and longitude.
    """
    zone = int((lon + 180) / 6) + 1
    if lat >= 0:
        return 32600 + zone  # Northern Hemisphere
    else:
        return 32700 + zone  # Southern Hemisphere


def download_labels(config_path, study_site, output_root):
    """
    Download Labelbox annotations for a study site, save shapefile,
    and return the shapefile path.
    """
    # Load config
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    api_key = config["api_key"]
    project_id = config["project_id"]

    client = Client(api_key)
    project = client.get_project(project_id)
    print(f"Connected to project: {project.name}")

    # Kick off export
    export_task = project.export_v2(params={
        "attachments": False,
        "metadata_fields": False,
        "data_row_details": True,
        "label_details": True,
    })

    print("Waiting for export to complete...")
    export_task.wait_until_done()

    if export_task.errors:
        raise Exception(f"Export failed: {export_task.errors}")

    # Export result is inline
    export_result = export_task.result
    print(f"Received {len(export_result)} rows from export")

    features = []

    for record in export_result:
        row_name = record.get("data_row", {}).get("global_key", "")
        if not row_name.startswith(study_site):
            continue

        projects = record.get("projects", {})
        for proj_id, proj_content in projects.items():
            for label in proj_content.get("labels", []):
                annotations = label.get("annotations", {}).get("objects", [])
                for ann in annotations:
                    if ann.get("annotation_kind") in ("GeoPolygon", "GeoPolyline"):
                        geom = ann["geojson"]
                        properties = {
                            "data_row_id": record.get("data_row", {}).get("id"),
                            "name": row_name,
                            "feature_id": ann.get("feature_id"),
                            "class_name": ann.get("name"),
                            "study_site": study_site
                        }
                        features.append({
                            "geometry": shape(geom),
                            "properties": properties
                        })

    if not features:
        raise Exception("No matching features found in export.")

    # Build GeoDataFrame in WGS84
    geoms = [f["geometry"] for f in features]
    props = [f["properties"] for f in features]
    gdf = gpd.GeoDataFrame(props, geometry=geoms, crs="EPSG:4326")

    # Save to Labelbox folder
    out_dir = os.path.join(
        output_root,
        f"{study_site} 50x50 km - PLD",
        f"{study_site} Lakes from Labelbox - Shapefile"
    )
    os.makedirs(out_dir, exist_ok=True)
    shp_path = os.path.join(out_dir, f"{study_site}_lakes.shp")
    gdf.to_file(shp_path, driver="ESRI Shapefile")
    print(f"Saved shapefile: {shp_path}")

    return shp_path


def buffer_shapefile_auto_utm(input_shp, buffer_meters=60):
    """
    Buffer polygons by meters using auto-selected UTM zone and
    save to a parallel 'Buffered' folder.
    """
    gdf = gpd.read_file(input_shp)

    # Get centroid to pick UTM zone
    centroid = gdf.unary_union.centroid
    lon, lat = centroid.x, centroid.y
    epsg = latlon_to_utm_epsg(lat, lon)
    print(f"Auto-selected EPSG:{epsg} (UTM Zone) for buffering")

    # Reproject to UTM
    gdf_proj = gdf.to_crs(epsg)

    # Buffer
    gdf_proj["geometry"] = gdf_proj.geometry.buffer(buffer_meters)

    # Reproject back to input CRS
    gdf_buffered = gdf_proj.to_crs(gdf.crs)

    # Construct output path: replace "Labelbox" with "Buffered"
    input_dir = os.path.dirname(input_shp)
    output_dir = input_dir.replace("from Labelbox", "Buffered")
    os.makedirs(output_dir, exist_ok=True)

    base_name = os.path.splitext(os.path.basename(input_shp))[0]
    output_shp = os.path.join(output_dir, f"{base_name}_buffer{buffer_meters}m.shp")

    # Save
    gdf_buffered.to_file(output_shp, driver="ESRI Shapefile")
    print(f"Saved buffered shapefile to {output_shp}")

    return output_shp


if __name__ == "__main__":

    # Step 1: Download from Labelbox
    shp_path = download_labels(config_path, study_site, output_root)

    # Step 2: Create buffered version in parallel "Buffered" folder
    buffered_shapefile = buffer_shapefile_auto_utm(shp_path, buffer_meters=60)

    print("Buffered Shapefile Path:", buffered_shapefile)

Connected to project: Lake Ice Project - Water Body Delineation


C:\Users\nj142\AppData\Local\Temp\ipykernel_40408\2591003859.py:36: DeprecationWarning: You are currently utilizing export_v2 for this action, which will be removed in 7.0. Please refer to our docs for export alternatives. https://docs.labelbox.com/reference/export-overview#export-methods
  export_task = project.export_v2(params={


Waiting for export to complete...
Received 3 rows from export
Saved shapefile: E:\planetscope_lake_ice\Data\Input\GR 50x50 km - PLD\GR Lakes from Labelbox - Shapefile\GR_lakes.shp


C:\Users\nj142\AppData\Local\Temp\ipykernel_40408\2591003859.py:95: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(shp_path, driver="ESRI Shapefile")
c:\Users\nj142\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'data_row_id' to 'data_row_i'
  ogr_write(
C:\Users\nj142\AppData\Local\Temp\ipykernel_40408\2591003859.py:109: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf.unary_union.centroid


Auto-selected EPSG:32622 (UTM Zone) for buffering
Saved buffered shapefile to E:\planetscope_lake_ice\Data\Input\GR 50x50 km - PLD\GR Lakes Buffered - Shapefile\GR_lakes_buffer60m.shp
Buffered Shapefile Path: E:\planetscope_lake_ice\Data\Input\GR 50x50 km - PLD\GR Lakes Buffered - Shapefile\GR_lakes_buffer60m.shp
